# Analyse prosopographique des philosophes nés après 1700

Ce notebook explore la distribution de variables qualitatives (genre, nationalité, école de pensée, formation universitaire) et leur évolution dans le temps à partir d’un extrait Wikidata de philosophes nés après 1700.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

plt.style.use('seaborn-v0_8')
sns.set_palette("pastel")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("philosophes_post1700.csv")

df.rename(columns={
    'birthDate': 'date_naissance',
    'sexLabel': 'genre',
    'movementLabel': 'ecole_pensee',
    'nationalityLabel': 'nationalite',
    'universityLabel': 'universite'
}, inplace=True)

df['date_naissance'] = pd.to_datetime(df['date_naissance'], errors='coerce')
df['annee'] = df['date_naissance'].dt.year
df['decennie'] = (df['annee'] // 10) * 10
df['siecle'] = ((df['annee'] - 1) // 100 + 1).astype("Int64")

for col in ['genre', 'ecole_pensee', 'nationalite', 'universite']:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace(['nan', 'None', ''], pd.NA)

df.head()

## Distribution par genre

In [ ]:
genre_counts = df["genre"].value_counts(dropna=False)
sns.barplot(x=genre_counts.index, y=genre_counts.values)
plt.title("Distribution du genre des philosophes")
plt.ylabel("Nombre")
plt.xlabel("Genre")
plt.show()

## Évolution du genre dans le temps

In [ ]:
sns.histplot(data=df, x="annee", hue="genre", multiple="stack", bins=30)
plt.title("Évolution du genre dans le temps")
plt.xlabel("Année de naissance")
plt.ylabel("Nombre")
plt.show()

## Distribution par nationalité

In [ ]:
nat_counts = df["nationalite"].value_counts().head(10)
sns.barplot(y=nat_counts.index, x=nat_counts.values)
plt.title("Top 10 des nationalités les plus représentées")
plt.xlabel("Nombre")
plt.ylabel("Nationalité")
plt.show()

## Évolution des nationalités principales

In [ ]:
temp_nat = df.groupby(["decennie", "nationalite"]).size().reset_index(name="effectif")
top_nat = df["nationalite"].value_counts().head(5).index
temp_nat = temp_nat[temp_nat["nationalite"].isin(top_nat)]

fig = px.line(temp_nat, x="decennie", y="effectif", color="nationalite",
              title="Évolution des nationalités principales dans le temps")
fig.show()

## Distribution des universités

In [ ]:
univ_counts = df["universite"].value_counts().head(10)
sns.barplot(y=univ_counts.index, x=univ_counts.values)
plt.title("Universités les plus fréquemment mentionnées")
plt.xlabel("Nombre")
plt.ylabel("Université")
plt.show()

## Distribution des écoles de pensée

In [ ]:
ecole_counts = df["ecole_pensee"].value_counts().head(10)
sns.barplot(y=ecole_counts.index, x=ecole_counts.values)
plt.title("Top 10 des écoles de pensée")
plt.xlabel("Nombre")
plt.ylabel("École de pensée")
plt.show()

## Évolution historique des écoles de pensée

In [ ]:
temp_ecole = df.groupby(["decennie", "ecole_pensee"]).size().reset_index(name="effectif")
top_ecoles = df["ecole_pensee"].value_counts().head(6).index
temp_ecole = temp_ecole[temp_ecole["ecole_pensee"].isin(top_ecoles)]

fig = px.line(temp_ecole, x="decennie", y="effectif", color="ecole_pensee",
              title="Évolution historique des écoles de pensée majeures")
fig.show()